In [ ]:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials# Authenticate and create the PyDrive client.
import pandas as pd

auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [ ]:
id = "1uiIMbheUVJK82LMshquFhDAQW8vAE6Dt"
downloaded = drive.CreateFile({'id':id}) 
downloaded.GetContentFile('gene.trainset.csv')  
train = pd.read_csv('gene.trainset.csv')

In [ ]:
gene_train = train.drop(columns = ["Cancer_Type", "Sample_ID"])

In [ ]:
gene_train.head

<bound method NDFrame.head of       ENSG00000141510  ENSG00000155657  ...  ENSG00000277957  ENSG00000258453
0            7.214132         0.042913  ...         0.023087              0.0
1            9.173340         0.037377  ...         0.068649              0.0
2            3.121146         0.144117  ...         0.010627              0.0
3            7.809700         0.087297  ...         0.000000              0.0
4           10.909652         0.064092  ...         0.014863              0.0
...               ...              ...  ...              ...              ...
5733        28.847554         0.105384  ...         0.031695              0.0
5734        21.335629         0.188096  ...         0.005672              0.0
5735        23.052332         0.014752  ...         0.010123              0.0
5736        20.974753         0.015865  ...         0.019282              0.0
5737        30.579034         0.088723  ...         0.030134              0.0

[5738 rows x 19420 columns]>

In [ ]:
id = "1MCvPmLzJnMq-DM9ch0MS7B3btv7RN3QO"
downloaded = drive.CreateFile({'id':id}) 
downloaded.GetContentFile('gene.val.csv')  
test = pd.read_csv('gene.val.csv')

In [ ]:
gene_test = test.drop(columns = ["Cancer_Type", "Sample_ID"])

In [ ]:
id = '1ZhpxOI75zZBBJMRu4supVZzWzGmoFUaR'
downloaded = drive.CreateFile({'id':id}) 
downloaded.GetContentFile('pro.trainset.csv')  
train = pd.read_csv('pro.trainset.csv')

In [ ]:
lable_train = list(pro1.Cancer_Type)
pro_train = pro1.drop(columns = ["Cancer_Type", "Sample_ID"])
print(len(set(lable_train)))

32


In [ ]:
id = '1Mof6HE8iPsGB5-ZzXvE8pulizup1EVjM'
downloaded = drive.CreateFile({'id':id}) 
downloaded.GetContentFile('pro.val.csv')  
test = pd.read_csv('pro.val.csv')

In [ ]:
lable_test = list(pro2.Cancer_Type)
pro_test = pro2.drop(columns = ["Cancer_Type", "Sample_ID"])

In [ ]:
from sklearn.preprocessing import LabelEncoder
import numpy as np

le = LabelEncoder()
lable1 = np.ravel(lable_train)
le.fit(lable1)
lable1 = le.transform(lable1)

le = LabelEncoder()
lable2 = np.ravel(lable_test)
le.fit(lable2)
lable2 = le.transform(lable2)


In [ ]:
import keras.utils as np_utils
cat1 = np_utils.to_categorical(lable1)
cat1 = pd.DataFrame({'tag' + str(i) : cat1[:, i]for i in range(len(cat1[0]))})
cat_train = cat1[:gene_train.shape[0]]
print(cat_train)

cat2 = np_utils.to_categorical(lable2)
cat2 = pd.DataFrame({'tag' + str(i) : cat2[:, i]for i in range(len(cat2[0]))})
cat_test = cat2[:gene_test.shape[0]]
print(cat_test)

      tag0  tag1  tag2  tag3  tag4  ...  tag27  tag28  tag29  tag30  tag31
0      1.0   0.0   0.0   0.0   0.0  ...    0.0    0.0    0.0    0.0    0.0
1      1.0   0.0   0.0   0.0   0.0  ...    0.0    0.0    0.0    0.0    0.0
2      1.0   0.0   0.0   0.0   0.0  ...    0.0    0.0    0.0    0.0    0.0
3      1.0   0.0   0.0   0.0   0.0  ...    0.0    0.0    0.0    0.0    0.0
4      1.0   0.0   0.0   0.0   0.0  ...    0.0    0.0    0.0    0.0    0.0
...    ...   ...   ...   ...   ...  ...    ...    ...    ...    ...    ...
5733   0.0   0.0   0.0   0.0   0.0  ...    0.0    0.0    0.0    0.0    1.0
5734   0.0   0.0   0.0   0.0   0.0  ...    0.0    0.0    0.0    0.0    1.0
5735   0.0   0.0   0.0   0.0   0.0  ...    0.0    0.0    0.0    0.0    1.0
5736   0.0   0.0   0.0   0.0   0.0  ...    0.0    0.0    0.0    0.0    1.0
5737   0.0   0.0   0.0   0.0   0.0  ...    0.0    0.0    0.0    0.0    1.0

[5738 rows x 32 columns]
     tag0  tag1  tag2  tag3  tag4  ...  tag27  tag28  tag29  tag30  tag31


In [ ]:
from keras.layers import Input, Dense, BatchNormalization, Lambda, GaussianNoise, GaussianDropout, Add
from keras.models import Model
import keras.backend as backend
import numpy as np
import keras
from sklearn.preprocessing import LabelEncoder, normalize
from keras.utils import np_utils
import pandas as pd
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
import time

Instructions for updating:
non-resource variables are not supported in the long term


In [ ]:
def model(m_rna_train, pro_train,label_train, m_rna_test, pro_test, label_test):
        opt = keras.optimizers.Nadam(learning_rate=1e-3)

        inputs = Input(shape=(m_rna_train.shape[1],), name="inputs")
        inputs_noise = GaussianNoise(stddev=0.25)(inputs)
        inputs_noise = GaussianDropout(rate=0.5 ** 2 / (1 + 0.5 ** 2))(inputs_noise)
        inputs_0 = BatchNormalization(name="inputs_0")(inputs_noise)
        inputs_1 = Dense(1024, activation="relu", name="inputs_1")(inputs_0)
        inputs_2 = BatchNormalization(name="inputs_2")(inputs_1)
        inputs_3 = Dense(256, activation="relu", name="inputs_3")(inputs_2)
        inputs_4 = BatchNormalization(name="inputs_4")(inputs_3)
        inputs_5 = Dense(64, activation="softplus", name="inputs_5")(inputs_4)
        inputs_6 = BatchNormalization(name="inputs_6")(inputs_5)

        encoded = Dense(units=20, activation='relu', name='encoded')(inputs_6)

        inputs_7 = Dense(64, activation="softplus", name="inputs_7")(encoded)
        inputs_8 = Dense(256, activation="relu", name="inputs_8")(inputs_7)
        # skip = Add()([inputs_8, inputs_3])
        # inputs_9 = Dense(1024, activation="relu", name="inputs_9")(skip)

        inputs_10 = Dense(128, activation='softplus', name = 'inputs_10')(encoded)
        cl_1 = Dense(32, activation = 'softmax', name= 'classifier')(encoded)
        decoded_tcga = Dense(units=m_rna_train.shape[1], activation='linear', name="m_rna")(inputs_9)
        decoded_micro_rna = Dense(units=pro_train.shape[1], activation='linear', name="pro")(inputs_10)

        scae = Model(inputs=inputs, outputs=[decoded_tcga, decoded_micro_rna, cl_1])
        lambda_value = 2.7849e-3


        def contractive_loss(y_pred, y_true):
            w = scae.get_layer('encoded').get_weights()[0] # N inputs N_hidden
            w = tf.transpose(w)  # N_hidden inputs N
            h = scae.get_layer('encoded').output
            dh = h * (1 - h)  # N_batch inputs N_hidden
            # N_batch inputs N_hidden * N_hidden inputs 1 = N_batch inputs 1
            contractive = lambda_value * backend.sum(dh ** 2 * backend.sum(w ** 2, axis=1), axis=1)
            mse = backend.mean(backend.square(y_true - y_pred), axis=1)
            return mse + contractive

        scae.compile(optimizer=opt,
                     loss=[contractive_loss, "mse", "categorical_crossentropy"],
                     loss_weights=[0.1, 0.05, 0.5],
                     metrics={"m_rna": ["mae", "mse"], "pro": ["mae", "mse"], "classifier":"acc"})
        
        history = scae.fit([m_rna_train], [m_rna_train, pro_train, label_train],
                 batch_size=128, epochs=200, verbose=1, validation_split=0.1, shuffle = True)
        score = scae.evaluate(m_rna_test, [m_rna_test, pro_test, label_test], verbose=1,
                              batch_size=128)
        pred_pro = scae.predict(m_rna_test, verbose=1,
                              batch_size=128)
        return pred_pro, score, history

In [ ]:
print(gene_train.shape, gene_test.shape)
print(pro_train.shape, pro_test.shape)

(5738, 19420) (673, 19420)
(5738, 216) (673, 216)


In [ ]:
train_x = gene_train.to_numpy()
test_x = gene_test.to_numpy()
train_y = pro_train.to_numpy()
test_y = pro_test.to_numpy()
label_train = np.array(cat_train)
label_test = np.array(cat_test)
pred_pro, score, history = model(train_x, train_y, label_train, test_x, test_y, label_test)

print(score)

Train on 5164 samples, validate on 574 samples
Epoch 1/200
5164/5164 [==============================] - ETA: 0s - loss: 750.0476 - m_rna_loss: 7490.1104 - pro_loss: 0.5391 - classifier_loss: 2.1506 - m_rna_mean_absolute_error: 15.3001 - m_rna_mean_squared_error: 7482.8232 - pro_mean_absolute_error: 0.5015 - pro_mean_squared_error: 0.5415 - classifier_acc: 0.5041

/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/engine/training.py:2325: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  warnings.warn('`Model.state_updates` will be removed in a future version. '


5164/5164 [==============================] - 64s 12ms/sample - loss: 750.0476 - m_rna_loss: 7490.1104 - pro_loss: 0.5391 - classifier_loss: 2.1506 - m_rna_mean_absolute_error: 15.3001 - m_rna_mean_squared_error: 7482.8232 - pro_mean_absolute_error: 0.5015 - pro_mean_squared_error: 0.5415 - classifier_acc: 0.5041 - val_loss: 450.4687 - val_m_rna_loss: 4531.0786 - val_pro_loss: 0.6016 - val_classifier_loss: 5.6170 - val_m_rna_mean_absolute_error: 20.8509 - val_m_rna_mean_squared_error: 4381.4575 - val_pro_mean_absolute_error: 0.5528 - val_pro_mean_squared_error: 0.6050 - val_classifier_acc: 0.0105
Epoch 2/200
5164/5164 [==============================] - 61s 12ms/sample - loss: 567.7782 - m_rna_loss: 5653.0166 - pro_loss: 0.4176 - classifier_loss: 2.1101 - m_rna_mean_absolute_error: 15.3457 - m_rna_mean_squared_error: 5661.6670 - pro_mean_absolute_error: 0.4246 - pro_mean_squared_error: 0.4176 - classifier_acc: 0.5033 - val_loss: 375.3694 - val_m_rna_loss: 3786.4382 - val_pro_loss: 0.4273

In [ ]:
p = pred_pro[1]
rank  = 0
for k in range(len(p)):
  temp = np.vstack((test_y, p[k]))
  for i in range(temp.shape[0]):
    temp[i] = np.subtract(temp[i], test_y[k])
  temp = np.square(temp)
  sum = temp.sum(axis = 1)
  arr = []
  for i in range(len(sum)):
    arr.append((sum[i], i))
  arr = sorted(arr, key=lambda x: x[0])
  for i in range(len(arr)):
    if arr[i][1] == len(p):
      rank += i
      break

rank /= len(p)
print(rank)

In [ ]:
import matplotlib.pyplot as plt

print(history.history['val_pro_mean_squared_error'][40])
plt.plot(history.history['classifier_acc'])
plt.plot(history.history['val_classifier_acc'])
plt.title('classification accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()
# summarize history for loss
plt.plot(history.history['pro_loss'])
plt.plot(history.history['val_pro_loss'])
plt.title('protein loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

plt.plot(history.history['m_rna_loss'])
plt.plot(history.history['val_m_rna_loss'])
plt.title('m RNA loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()